### 데이터 분석 환경 설정

In [1]:
import os
import sys
import pandas as pd
os.environ['JAVA_HOME'] = 'C:\java'
os.environ['SPARK_HOME'] = "C:\spark-3.3.2"
os.environ['PYLIB'] = "C:\spark-3.3.2\python\lib"
sys.path.insert(0,os.environ['PYLIB']+"\py4j-0.10.9.5-scr.zip")
sys.path.insert(0,os.environ['PYLIB']+"\pyspark.zip")

####  spark 경로를 찾지 못하는 경우, findspark설치

In [2]:
# !pip install findspark
import findspark
findspark.find()
findspark.init(os.environ.get("SPARK_HOME"))

#### pyspark 스파크 생성

In [3]:
import pyspark
from pyspark.sql import SparkSession
스파크 =  SparkSession.builder.appName('Test').getOrCreate()
스파크.conf.set("spark.sql.execution.arrow.enabled","true")
스파크

### pyspark에서 hdfs 데이터 가져오기

#### hdfs데이터를 가져와 총 7개의 스파크 데이터 프레임 생성하기

In [4]:
df1 = 스파크.read.parquet("hdfs://localhost:9000/finalDB/Partial1.parquet")

In [5]:
df2 = 스파크.read.parquet("hdfs://localhost:9000/finalDB/Partial2.parquet")

In [6]:
df3 = 스파크.read.option('header','true').csv("hdfs://localhost:9000/finalDB/Partial3.csv",inferSchema=True)

In [7]:
df4 = 스파크.read.parquet("hdfs://localhost:9000/finalDB/Partial4.parquet")

In [8]:
df5 = 스파크.read.option('header', 'true').csv("hdfs://localhost:9000/finalDB/Partial5.csv",inferSchema=True)

In [9]:
testX = 스파크.read.option('header', 'true').csv("hdfs://localhost:9000/finalDB/TestX.csv",inferSchema=True)

In [10]:
testY= 스파크.read.option('header', 'true').csv("hdfs://localhost:9000/finalDB/TestY.csv",inferSchema=True)

### 데이터 탐색-Pyspark EDA

#### df1~df5를   "ID"를 기준으로 병합하여 최종적으로 "df_final" pyspark 데이터프레임 생성

##### 모두 같은 칼럼을 가진 df1,df2,df3을 위아래로  중복 없이 병합 한  "df_left"데이터프레임 생성

In [11]:
df_left = df1.union(df2).union(df3).distinct()
df_left.show()

+--------+---+-------+------+-------+------+----------------+
|      ID| X1|     X2|    X3|     X4|    X5|               Y|
+--------+---+-------+------+-------+------+----------------+
|P1004643| A1| 0.8241|14.545| 7.2912|Group2|9313.76853203857|
|P1005855| A1|-0.6758|11.204|11.7577|Group2|36483.4964909969|
|P1006113| A1|-1.4719|17.351| 5.5188|Group2|15842.8529367417|
|P1006987| A2|-0.0909|-6.145| 8.3346|Group2|305642.810408734|
|P1007068| A2|-0.1845|-7.694|13.0823|Group3|870.308915074029|
|P1014849| A2| 1.0198| 2.507| 9.4321|Group2|51866.5751369749|
|P1015020| A3| 1.8171|11.937| 10.505|Group2|263892.767700988|
|P1017689| A2|-0.1643|29.753|10.6848|Group2|19217.5939501822|
|P1023051| A4|-1.1166|12.803| 8.5952|Group2|7395.73514236024|
|P1024352| A1| 0.4547|  7.59|  8.751|Group2|1613082.13890887|
|P1026978| A4|-1.0673| 2.968| 9.6272|Group1|6330.48003368929|
|P1039605| A2|-0.4541|21.769| 6.5469|Group2|133742.581589266|
|P1039731| A3|-1.1484|31.053|10.2961|Group2|35834.7151739424|
|P104049

##### 생성한 df_left 와 df4데이터 프레임을 "ID"를 기준으로 중복없이 좌우로 합병하여 df_top 데이터프레임 생성

In [12]:
## 먼저 df_left의 칼럼명 변경 : df_4와 합병했을 때 중복된 칼럼을 방지하기 위해서
df_left = df_left.withColumnRenamed('ID', 'ID_L')

##df_left와 df4 데이터 합병하여  df_top 데이터 프레임 생성
df_top = df_left.join(df4, df_left.ID_L == df4.ID)

## 칼럼 ID와 ID_L은 같은 값을 가지니까 , "ID_L" 칼럼 삭제
df_top = df_top.drop('ID_L')

## df_top과 df5의 칼럼 스키마를 똑같이 만들기 위해서 df_top의 칼럼 위치 조정
df_top = df_top.select('ID', 'X1','X2','X3', 'X4','X5','X6','X7','X8', 'X9','X10','Y' )
df_top.show()

+--------+---+-------+------+-------+------+----+----+--------+---+------+----------------+
|      ID| X1|     X2|    X3|     X4|    X5|  X6|  X7|      X8| X9|   X10|               Y|
+--------+---+-------+------+-------+------+----+----+--------+---+------+----------------+
|P1000002| A3| 0.1972| 6.563| 8.4542|Group3| 5.0|Age4|10.60235| 82|  Male|27612.3830690663|
|P1000060| A1| 0.4274|17.689|10.6112|Group2| 6.0|Age4| 9.90083|115|Female|148313.339805297|
|P1000084| A2|-1.1741|  4.27|10.4408|Group2| 3.0|Age3|10.09016| 69|Female|3065.22776267054|
|P1000134| A3|-0.3956|17.278| 9.9707|Group2|null|Age3| 9.90555| 87|Female|17878.3668202395|
|P1000225| A2|-0.4578|-0.326|10.1997|Group1| 7.0|Age4| 9.89995| 75|  Male|25117.4149775698|
|P1000235| A2|-0.4115|-5.716| 9.2232|Group2| 5.0|Age4| 9.07033|128|  null|43666.8277002937|
|P1000277| A4| 0.2783|10.754| 10.626|Group2| 5.0|Age3|12.54388|100|  Male| 3590.2300979165|
|P1000318| A2|-1.0334|10.578| 8.5691|Group3| 5.0|Age3| 8.45483| 80|  Male|28899.

 ##### 생성한 df_top 와 df5데이터 프레임을 "ID"를 기준으로 중복없이 위아래로 합병하여,  최종적으로 "df_final"  데이터프레임 생성

In [13]:
## 두개의 데이터프레임 중복없이 위아래로 병합
df_final = df_top.union(df5).distinct()
df_final.show()

+--------+---+-------+-------+-------+------+----+----+--------+---+------+----------------+
|      ID| X1|     X2|     X3|     X4|    X5|  X6|  X7|      X8| X9|   X10|               Y|
+--------+---+-------+-------+-------+------+----+----+--------+---+------+----------------+
|P1011425| A1| 0.1425| 24.421| 8.5743|Group2| 4.0|Age4|10.92726|112|  Male|40524.7689561014|
|P1011567| A2| 0.4552| -5.375|11.6004|Group2| 4.0|Age1|11.68286|112|  Male|1472.59588701139|
|P1016416| A4|-2.1239|  9.722| 8.5259|Group2| 5.0|Age2| 9.17738|100|Female|1144.21742749272|
|P1027231| A2| 0.4323|  14.28|10.0368|Group1| 7.0|Age2| 9.06071| 96|  Male|57248.4634773109|
|P1027405| A1| 1.4931|  9.445|    6.5|Group1| 5.0|Age5|10.35746|104|Female|895958.082217849|
|P1030907| A3| 0.7866| 19.784| 6.3409|Group2| 6.0|Age2| 9.88512|115|Female|70327.1923723186|
|P1031209| A1|-0.7524|  6.257| 8.4731|Group2| 5.0|Age2| 9.23465|118|Female|13658.9099910543|
|P1032106| A2|-0.7758| 17.449|13.2908|Group2| 6.0|Age2| 8.39832| 66|  

 ##### 3.1.3  testX와 testY  "ID"를 기준으로 합병하여, 최종적으로 test란 "테스트 데이터프레임" 생성

In [14]:
## 먼저 testX의 칼럼명 변경 : df_4와 합병했을 때 중복된 칼럼을 방지하기 위해서
testX = testX.withColumnRenamed('ID', 'ID_L')

##testX와 testY 데이터 합병하여  test 데이터 프레임 생성
test = testX.join(testY, testX.ID_L == testY.ID)
test

DataFrame[ID_L: string, X1: string, X2: double, X3: double, X4: string, X5: string, X6: string, X7: string, X8: double, X9: int, X10: string, ID: string, Y: double]

In [15]:
test= test.drop('ID_L')
test = test.select('ID', 'X1','X2','X3', 'X4','X5','X6','X7','X8', 'X9','X10','Y' )
test.show()

+--------+---+-------+-------+-------+------+---+----+--------+---+------+----------------+
|      ID| X1|     X2|     X3|     X4|    X5| X6|  X7|      X8| X9|   X10|               Y|
+--------+---+-------+-------+-------+------+---+----+--------+---+------+----------------+
|P9587410| A4| 1.4451| 21.091| 9.8848|Group1|  5|Age5|10.09841| 92|  Male| 1002255.7947277|
|P9587424| A1|-0.0209| 13.235| 7.8846|Group2|  5|Age3|10.37013| 99|Female|15288.0881575988|
|P9587426| A3| 0.7792| 17.257|10.6058|Group2|  5|Age6| 9.56067| 95|  Male|4120015.07658163|
|P9587439| A2|-0.1679| 31.393| 12.692|Group2|  5|Age4|10.63859| 98|Female|56796.1642327521|
|P9587448| A1| 0.8661|  7.413|     NA|Group3|  6|Age2| 7.36374| 87|  Male|228256.069581107|
|P9587472| A4|-1.3435| 28.384| 9.2535|Group2|  5|Age4| 9.41166|102|  Male|25150.8616553314|
|P9587487| A3|-0.6942|  5.113|11.4571|Group3|  4|Age5| 10.5509|101|Female|19113.5291888059|
|P9587504| A1|-0.6296|  0.812|10.1179|Group1|  5|Age6| 9.37954| 83|  Male| 10191

#### 훈련 데이터인 "df_final" 과 테스트 데이터인 "test" 데이터 탐색

In [16]:
df_final.printSchema()

root
 |-- ID: string (nullable = true)
 |-- X1: string (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: string (nullable = true)
 |-- X5: string (nullable = true)
 |-- X6: string (nullable = true)
 |-- X7: string (nullable = true)
 |-- X8: double (nullable = true)
 |-- X9: integer (nullable = true)
 |-- X10: string (nullable = true)
 |-- Y: double (nullable = true)



In [17]:
test.printSchema()

root
 |-- ID: string (nullable = true)
 |-- X1: string (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: string (nullable = true)
 |-- X5: string (nullable = true)
 |-- X6: string (nullable = true)
 |-- X7: string (nullable = true)
 |-- X8: double (nullable = true)
 |-- X9: integer (nullable = true)
 |-- X10: string (nullable = true)
 |-- Y: double (nullable = true)



##### df_final, test  를 각각 판다스 프레임 pd_ train, pd_test  으로 변환 후 데이터 탐색 

In [18]:
#앞으로 train 데이터: pd_train, test 데이터: pd_test
pd_train = df_final.toPandas()
pd_test =test.toPandas()

C:\spark-3.3.2\python\pyspark\sql\pandas\conversion.py:114: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)
C:\spark-3.3.2\python\pyspark\sql\pandas\conversion.py:114: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 1.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


### 데이터 전처리 및 변수 변환

#### pd_train  , pd_test 결측치 확인 및 처리

In [19]:
print(pd_train.isna().sum())
print("\n")
print(pd_test.isna().sum())

ID         0
X1      4814
X2         0
X3         0
X4     18399
X5       938
X6     61524
X7         0
X8         0
X9         0
X10    21976
Y          0
dtype: int64


ID     0
X1     0
X2     0
X3     0
X4     0
X5     0
X6     0
X7     0
X8     0
X9     0
X10    0
Y      0
dtype: int64


In [20]:
## 결측치 있는 행 모두 제거
pd_train = pd_train.dropna(subset=['X1', 'X4', 'X5', 'X6','X10'])
print(pd_train.isna().sum())

ID     0
X1     0
X2     0
X3     0
X4     0
X5     0
X6     0
X7     0
X8     0
X9     0
X10    0
Y      0
dtype: int64


#### pd_train  , pd_test 컬럼 범주 확인 후 'NA'값을 가진 행 모두 삭제

In [21]:
##1) 'X1'변수 
print(pd_train['X1'].value_counts())
index = pd_train[pd_train['X1'] == 'NA'].index
pd_train = pd_train.drop(index)
print("\n")
print("변수 전환 후:", pd_train['X1'].value_counts())
print("\n")

print(pd_test['X1'].value_counts())
index = pd_test[pd_test['X1'] == 'NA'].index
pd_test = pd_test.drop(index)
print("\n")
print("변수 전환 후:", pd_test['X1'].value_counts())

A1    248046
A4    247887
A2    247745
A3    247582
NA      5479
Name: X1, dtype: int64


변수 전환 후: A1    248046
A4    247887
A2    247745
A3    247582
Name: X1, dtype: int64


A3    12065
A4    12063
A2    12035
A1    11960
NA      477
Name: X1, dtype: int64


변수 전환 후: A3    12065
A4    12063
A2    12035
A1    11960
Name: X1, dtype: int64


In [22]:
##2) 'X2' 변수
print(pd_train['X2'].value_counts())
index = pd_train[pd_train['X2'] == 'NA'].index
pd_train = pd_train.drop(index)
print("\n 변수 전환 후 ", pd_train['X2'].value_counts())
print("\n")

print(pd_test['X2'].value_counts())
index = pd_test[pd_test['X2'] == 'NA'].index
pd_test = pd_test.drop(index)
print("\n 변수 전환 후 ", pd_test['X2'].value_counts())

-0.5248    70
 0.1088    65
-0.1304    64
 0.4033    64
 0.1939    63
           ..
-2.4832     1
 3.9187     1
-2.6587     1
 2.9168     1
 3.4740     1
Name: X2, Length: 57113, dtype: int64

 변수 전환 후  -0.5248    70
 0.1088    65
-0.1304    64
 0.4033    64
 0.1939    63
           ..
-2.4832     1
 3.9187     1
-2.6587     1
 2.9168     1
 3.4740     1
Name: X2, Length: 57113, dtype: int64


 0.0732    9
 0.1208    8
-0.1692    8
 0.3283    8
 0.3124    8
          ..
 1.2890    1
-0.5168    1
-0.4498    1
 0.6231    1
-1.6316    1
Name: X2, Length: 27334, dtype: int64

 변수 전환 후   0.0732    9
 0.1208    8
-0.1692    8
 0.3283    8
 0.3124    8
          ..
 1.2890    1
-0.5168    1
-0.4498    1
 0.6231    1
-1.6316    1
Name: X2, Length: 27334, dtype: int64


In [23]:
## 3) 'X4'

print(pd_train['X4'].value_counts())
index = pd_train[pd_train['X4'] == 'NA'].index
pd_train = pd_train.drop(index)
print("\n 변수전환 후 ",pd_train['X4'].value_counts() )
print("\n")

print(pd_test['X4'].value_counts())
index = pd_test[pd_test['X4'] == 'NA'].index
pd_test = pd_test.drop(index)
print("\n 변수 전환 후 ",pd_test['X4'].value_counts() )

NA         20757
10.2894       40
10.2601       39
10.0048       38
11.2662       38
           ...  
6.0248         1
15.8131        1
5.3801         1
15.1429        1
4.9204         1
Name: X4, Length: 103277, dtype: int64

 변수전환 후  10.2894    40
10.2601    39
11.1933    38
11.2662    38
10.0048    38
           ..
5.668       1
13.6932     1
14.551      1
16.3957     1
4.9204      1
Name: X4, Length: 103276, dtype: int64


NA         1841
10.786        6
11.4878       6
9.4315        6
8.9312        6
           ... 
9.328         1
12.4488       1
9.8127        1
12.8287       1
8.054         1
Name: X4, Length: 34493, dtype: int64

 변수 전환 후  9.4315     6
11.4878    6
8.6579     6
9.7915     6
8.9312     6
          ..
10.0622    1
9.8175     1
14.3815    1
12.5557    1
8.054      1
Name: X4, Length: 34492, dtype: int64


In [24]:
##4) 'X5' 변수
print(pd_train['X5'].value_counts())
index = pd_train[pd_train['X5'] == 'NA'].index
pd_train = pd_train.drop(index)
print("\n 변수 전환 후 ", pd_train['X5'].value_counts())
print("\n")

print(pd_test['X5'].value_counts())
index = pd_test[pd_test['X5'] == 'NA'].index
pd_test = pd_test.drop(index)
print("\n 변수 전환 후 ", pd_test['X5'].value_counts())

Group2    662819
Group1    153392
Group3    153267
NA          1025
Name: X5, dtype: int64

 변수 전환 후  Group2    662819
Group1    153392
Group3    153267
Name: X5, dtype: int64


Group2    31468
Group3     7389
Group1     7327
NA           98
Name: X5, dtype: int64

 변수 전환 후  Group2    31468
Group3     7389
Group1     7327
Name: X5, dtype: int64


In [25]:
##5) 'X6' 변수
print(pd_train['X6'].value_counts())
index = pd_train[pd_train['X6'] == 'NA'].index
pd_train = pd_train.drop(index)
print("\n 변수 전환 후 ",pd_train['X6'].value_counts() )
print("\n")

print(pd_test['X6'].value_counts())
index = pd_test[pd_test['X6'] == 'NA'].index
pd_test = pd_test.drop(index)
print("\n 변수 전환 후",pd_test['X6'].value_counts() )

5.0     262019
6.0     165402
4.0     164958
5       100646
6        64096
4        63454
7.0      41534
3.0      41292
NA       22156
3        16109
7        15882
8.0       4130
2.0       4064
8         1705
2         1559
1.0        174
9.0        157
9           70
1           65
10.0         4
0.0          2
Name: X6, dtype: int64

 변수 전환 후  5.0     262019
6.0     165402
4.0     164958
5       100646
6        64096
4        63454
7.0      41534
3.0      41292
3        16109
7        15882
8.0       4130
2.0       4064
8         1705
2         1559
1.0        174
9.0        157
9           70
1           65
10.0         4
0.0          2
Name: X6, dtype: int64


5     16389
6     10318
4     10209
NA     3591
7      2601
3      2538
2       264
8       254
9        11
1         8
0         1
Name: X6, dtype: int64

 변수 전환 후 5    16389
6    10318
4    10209
7     2601
3     2538
2      264
8      254
9       11
1        8
0        1
Name: X6, dtype: int64


In [26]:
##4) 'X10' 변수
print(pd_train['X10'].value_counts())
index = pd_train[pd_train['X10'] == 'NA'].index
pd_train = pd_train.drop(index)
print("\n 변수 전환 후 ", pd_train['X10'].value_counts())
print("\n")

print(pd_test['X10'].value_counts())
index = pd_test[pd_test['X10'] == 'NA'].index
pd_test = pd_test.drop(index)
print("\n 변수 전환 후", pd_test['X10'].value_counts())

Male      470232
Female    469847
NA          7243
Name: X10, dtype: int64

 변수 전환 후  Male      470232
Female    469847
Name: X10, dtype: int64


Female    20810
Male      20616
NA         1167
Name: X10, dtype: int64

 변수 전환 후 Female    20810
Male      20616
Name: X10, dtype: int64


####  pd_train, pd_test  데이터프레임 변수 변환

In [27]:
### pd_train 데이터 프레임  수치형 데이터 분포 확인
pd_train.describe()

,X2,X3,X8,X9,Y
count,940079.000000,940079.000000,940079.000000,940079.000000,9.400790e+05
mean,0.001134,9.999191,10.000299,97.301481,3.093611e+05
std,1.001569,9.996734,1.000544,19.988774,2.580399e+06
min,-5.091300,-36.183000,5.026390,0.000000,2.718282e+00
25%,-0.674300,3.258000,9.324020,84.000000,9.286259e+03
50%,0.001600,9.999000,10.000620,97.000000,3.732305e+04
75%,0.677000,16.741000,10.675925,111.000000,1.490385e+05
max,4.696400,57.621000,14.596300,192.000000,1.318816e+09


In [28]:
##pd_train, pd_test 데이터프레임 타입 확인
print(pd_train.dtypes)
print("\n")
print(pd_test.dtypes)

ID      object
X1      object
X2     float64
X3     float64
X4      object
X5      object
X6      object
X7      object
X8     float64
X9       int32
X10     object
Y      float64
dtype: object


ID      object
X1      object
X2     float64
X3     float64
X4      object
X5      object
X6      object
X7      object
X8     float64
X9       int32
X10     object
Y      float64
dtype: object


In [29]:
### 1) pd_train, pd_testX의 칼럼 'X4'와 'X6' 데이터 타입 변경하기
pd_train['X4'] = pd_train['X4'].astype('float64')
pd_test['X4'] = pd_test['X4'].astype('float64')

pd_train['X6'] = pd_train['X6'].astype('float64')
pd_test['X6'] = pd_test['X6'].astype('float64')

In [30]:
### 타입 변경 후'X4'와 'X6' 확인
pd_train[['X4', 'X6']].dtypes

X4    float64
X6    float64
dtype: object

In [31]:
pd_test[['X4', 'X6']].dtypes

X4    float64
X6    float64
dtype: object

In [32]:
### 2) pd_train, pd_testX의  명목형 변수를 수치형 변수로 변경 => 가변수 처리

### 2-1) 먼저 pd_final, pd_testX의 object 타입인 'X1', 'X5', 'X7','X9', 'X10' 범주 확인하기
print(pd_train['X1'].value_counts(), "\n") ##  1,2,3,4
print(pd_train['X5'].value_counts(), "\n") ##  1,2,3
print(pd_train['X7'].value_counts(), "\n") ##  1,2,3,4,5,6,
print(pd_train['X9'].value_counts(), "\n") ##  A,B,C,D,E,F,G
print(pd_train['X10'].value_counts(), "\n")##  M , F
print("*******")
print(pd_test['X1'].value_counts(), "\n") ##  1,2,3,4
print(pd_test['X5'].value_counts(), "\n") ##  1,2,3
print(pd_test['X7'].value_counts(), "\n") ##  1,2,3,4,5,6,
print(pd_test['X9'].value_counts(), "\n") ##  A,B,C,D,E,F,G
print(pd_test['X10'].value_counts(), "\n")##  M , F

A1    235319
A4    235159
A2    234813
A3    234788
Name: X1, dtype: int64 

Group2    642550
Group1    148781
Group3    148748
Name: X5, dtype: int64 

Age2    188211
Age4    187986
Age3    187920
Age5    187890
Age1     94120
Age6     93952
Name: X7, dtype: int64 

99     18811
96     18796
94     18676
98     18671
97     18635
       ...  
188        1
7          1
187        1
15         1
192        1
Name: X9, Length: 183, dtype: int64 

Male      470232
Female    469847
Name: X10, dtype: int64 

*******
A2    10402
A3    10369
A4    10349
A1    10306
Name: X1, dtype: int64 

Group2    28250
Group3     6630
Group1     6546
Name: X5, dtype: int64 

Age3    8399
Age4    8321
Age5    8238
Age2    8191
Age6    4227
Age1    4050
Name: X7, dtype: int64 

99    846
95    839
92    824
94    823
97    814
     ... 
12      1
31      1
16      1
19      1
28      1
Name: X9, Length: 150, dtype: int64 

Female    20810
Male      20616
Name: X10, dtype: int64 



In [33]:
###2-3)  pd_train, pd_test 가변수 처리
pd_train = pd.get_dummies(pd_train,columns=["X1","X5", "X7", "X10"],drop_first=True)
pd_test = pd.get_dummies(pd_test,columns=["X1","X5", "X7","X10"],drop_first=True)

In [34]:
### 2- 4) 마지막으로 pd_train, pd_test 칼럼 위치 정리 및 칼럼 명 수정

pd_train = pd_train.drop(['ID', 'X9'], axis = 1)
pd_train = pd_train[['X1_A2','X1_A3','X1_A4' , 'X2', 'X3', 'X4', 'X5_Group2','X5_Group3', 'X6', 'X7_Age2','X7_Age3','X7_Age4', 'X7_Age5','X7_Age6','X8','X10_Male','Y']]
pd_train.columns = ['A2','A3','A4' , 'X2', 'X3', 'X4', 'Group2','Group3', 'X6', 'Age2','Age3','Age4', 'Age5','Age6','X8','Male', 'Y']
pd_train.head(5)

,A2,A3,A4,X2,X3,X4,Group2,Group3,X6,Age2,Age3,Age4,Age5,Age6,X8,Male,Y
0,0,0,0,0.1425,24.421,8.5743,1,0,4.0,0,0,1,0,0,10.92726,1,40524.768956
1,1,0,0,0.4552,-5.375,11.6004,1,0,4.0,0,0,0,0,0,11.68286,1,1472.595887
2,0,0,1,-2.1239,9.722,8.5259,1,0,5.0,1,0,0,0,0,9.17738,0,1144.217427
3,1,0,0,0.4323,14.280,10.0368,0,0,7.0,1,0,0,0,0,9.06071,1,57248.463477
4,0,0,0,1.4931,9.445,6.5000,0,0,5.0,0,0,0,1,0,10.35746,0,895958.082218


In [35]:
pd_train.tail(10)

,A2,A3,A4,X2,X3,X4,Group2,Group3,X6,Age2,Age3,Age4,Age5,Age6,X8,Male,Y
1099990,0,0,0,1.8187,28.239,9.4684,1,0,5.0,0,1,0,0,0,10.07882,1,425872.097340
1099991,1,0,0,1.0645,14.740,8.0473,1,0,6.0,0,1,0,0,0,9.76764,0,116969.297302
1099992,1,0,0,0.9292,4.414,8.8902,1,0,4.0,0,0,0,1,0,10.17348,1,124799.457733
1099993,0,1,0,-0.5837,14.750,12.2788,1,0,5.0,1,0,0,0,0,10.84641,1,3244.466010
1099994,0,0,1,1.4074,5.339,8.2439,0,1,6.0,0,1,0,0,0,10.28944,0,129393.343655
1099995,0,0,0,-1.0282,17.681,11.3434,1,0,5.0,0,0,1,0,0,11.27901,0,4618.866227
1099996,1,0,0,0.4549,-1.109,10.6859,0,0,7.0,0,0,1,0,0,9.46518,0,61760.906250
1099997,0,0,0,-0.5986,4.344,8.4179,1,0,3.0,0,1,0,0,0,9.28151,1,21687.600470
1099998,0,1,0,1.2848,15.587,9.4074,1,0,7.0,1,0,0,0,0,9.28443,1,200976.711497
1099999,0,0,1,1.2989,-1.192,8.7882,0,0,4.0,0,0,0,0,0,11.72923,0,1981.223104


In [36]:
pd_test = pd_test.drop(['ID','X9'], axis = 1) 
pd_test = pd_test[['X1_A2','X1_A3','X1_A4' , 'X2', 'X3', 'X4', 'X5_Group2','X5_Group3', 'X6', 'X7_Age2','X7_Age3','X7_Age4', 'X7_Age5','X7_Age6','X8','X10_Male','Y']]
pd_test.columns = ['A2','A3','A4' , 'X2', 'X3', 'X4', 'Group2','Group3', 'X6', 'Age2','Age3','Age4', 'Age5','Age6','X8','Male', 'Y']
pd_test.head(5)

,A2,A3,A4,X2,X3,X4,Group2,Group3,X6,Age2,Age3,Age4,Age5,Age6,X8,Male,Y
0,0,0,1,1.4451,21.091,9.8848,0,0,5.0,0,0,0,1,0,10.09841,1,1.002256e+06
1,0,0,0,-0.0209,13.235,7.8846,1,0,5.0,0,1,0,0,0,10.37013,0,1.528809e+04
2,0,1,0,0.7792,17.257,10.6058,1,0,5.0,0,0,0,0,1,9.56067,1,4.120015e+06
3,1,0,0,-0.1679,31.393,12.6920,1,0,5.0,0,0,1,0,0,10.63859,0,5.679616e+04
5,0,0,1,-1.3435,28.384,9.2535,1,0,5.0,0,0,1,0,0,9.41166,1,2.515086e+04


In [38]:
### 3) 반응변수인 pd_train, pd_test의 'Y' 의 형태가 연속형 이므로 pd_train.median()의 값과 유사한 37000기준으로 0,1 의 값으로 나눔

pd_train['Y'].loc[(pd_train['Y'] <= 37000)] = 0
pd_train['Y'].loc[(pd_train['Y'] > 37000)] = 1

pd_test['Y'].loc[(pd_test['Y'] <= 37000)] = 0
pd_test['Y'].loc[(pd_test['Y'] > 37000)] = 1

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17412\3990477497.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_train['Y'].loc[(pd_train['Y'] > 37000)] = 1
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17412\3990477497.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_test['Y'].loc[(pd_test['Y'] > 37000)] = 1


In [39]:
pd_train['Y'] = pd_train['Y'].astype(int)
pd_test['Y'] = pd_test['Y'].astype(int)

In [40]:
pd_test

,A2,A3,A4,X2,X3,X4,Group2,Group3,X6,Age2,Age3,Age4,Age5,Age6,X8,Male,Y
0,0,0,1,1.4451,21.091,9.8848,0,0,5.0,0,0,0,1,0,10.09841,1,1
1,0,0,0,-0.0209,13.235,7.8846,1,0,5.0,0,1,0,0,0,10.37013,0,0
2,0,1,0,0.7792,17.257,10.6058,1,0,5.0,0,0,0,0,1,9.56067,1,1
3,1,0,0,-0.1679,31.393,12.6920,1,0,5.0,0,0,1,0,0,10.63859,0,1
5,0,0,1,-1.3435,28.384,9.2535,1,0,5.0,0,0,1,0,0,9.41166,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48595,0,0,0,-0.9022,25.929,11.0598,0,0,6.0,0,0,1,0,0,10.35923,0,0
48596,0,1,0,0.9984,18.744,8.0540,0,0,6.0,0,0,0,1,0,9.96515,0,1
48597,0,0,0,1.0163,4.117,10.5752,1,0,7.0,0,1,0,0,0,9.72330,1,1
48598,1,0,0,-1.3081,3.577,12.1787,0,0,6.0,1,0,0,0,0,10.17985,1,0


#### 설명 변수 선택 

In [41]:
###1) 유의미한 설명변수를 선택하기 위해 100여개의 자료를 랜덤 추출하여 유의성 검정 4번 실시

###1.1) pd_train 샘플을 추출하여 유의성 검정

pd_train_sample = pd_train.sample(n=1000, 
          replace=True) 
pd_train_sample

,A2,A3,A4,X2,X3,X4,Group2,Group3,X6,Age2,Age3,Age4,Age5,Age6,X8,Male,Y
1038049,1,0,0,-0.2740,18.303,10.3514,0,0,3.0,0,0,0,1,0,9.96887,0,1
593910,0,0,1,0.0660,8.109,9.1432,1,0,6.0,0,1,0,0,0,11.49179,1,0
1021485,0,0,0,-0.3362,-1.329,10.1103,0,1,4.0,0,1,0,0,0,9.96058,0,0
463636,0,0,0,0.8922,10.425,8.4509,1,0,5.0,0,0,0,1,0,9.88505,0,1
27611,0,0,1,-1.0782,3.295,8.4523,1,0,5.0,1,0,0,0,0,10.03565,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160890,1,0,0,-0.9242,16.369,10.8086,1,0,5.0,0,0,0,0,0,9.69689,1,0
702527,0,0,1,0.5705,3.910,13.4929,0,1,6.0,0,0,1,0,0,9.36742,1,1
454982,0,1,0,0.6284,15.868,8.7587,1,0,5.0,0,0,1,0,0,9.51430,0,1
676011,0,0,1,1.5851,22.104,14.4005,1,0,4.0,0,1,0,0,0,10.40433,1,1


In [42]:
pd_train_trainX = pd_train_sample[['A2','A3','A4' , 'X2', 'X3', 'X4', 'Group2','Group3', 'X6', 'Age2','Age3','Age4', 'Age5','Age6','X8','Male']]
pd_train_trainY = pd_train_sample[['Y']]

In [43]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
X = pd_train_trainX
y = pd_train_trainY['Y']

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [45]:
lrm = LinearRegression(n_jobs=-1)    
result = lrm.fit(X_train, y_train) 
print(result.coef_)

[-0.01937853  0.00790998  0.00688526  0.25737456  0.00494171  0.00138047
 -0.06425973  0.00610569 -0.00532937  0.16006755  0.34905856  0.48143681
  0.66349797  0.74212212 -0.17897933 -0.00446722]


In [46]:
print(result.feature_names_in_)

['A2' 'A3' 'A4' 'X2' 'X3' 'X4' 'Group2' 'Group3' 'X6' 'Age2' 'Age3' 'Age4'
 'Age5' 'Age6' 'X8' 'Male']


In [47]:
forecast = lrm.predict(X_test)
print(forecast[0:10])

[ 0.40870938 -0.18069331  0.25532994  0.38271465  0.13822499  0.64548177
  0.78432399 -0.41033751  1.46917175  0.92779277]


In [48]:
accuracy = lrm.score(X_test, y_test)
print(accuracy)

0.6380096877343179


In [49]:
X_train.dtypes

A2          uint8
A3          uint8
A4          uint8
X2        float64
X3        float64
X4        float64
Group2      uint8
Group3      uint8
X6        float64
Age2        uint8
Age3        uint8
Age4        uint8
Age5        uint8
Age6        uint8
X8        float64
Male        uint8
dtype: object

In [53]:
###유의성 검정 -4번 반복
import statsmodels.api as sm
model = sm.OLS(y_train, sm.add_constant(X_train)) 
results = model.fit()  
print(results.summary()) 

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.639
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                     75.64
Date:                Fri, 16 Jun 2023   Prob (F-statistic):          3.19e-139
Time:                        20:13:49   Log-Likelihood:                -150.19
No. Observations:                 700   AIC:                             334.4
Df Residuals:                     683   BIC:                             411.7
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9055      0.152     12.554      0.0

In [54]:
#####위의 총 4번의 유의성 검정에서 p값이 낮은 변수들이 유의하다고 판단,따라서 p값이 낮은 설명변수들을 선택

In [55]:
### 위의 분석표를 본 후  p값에 의해 유의한 변수 선택 
설명변수 = ['X2','X3','Age2','Age3','Age4','Age5','Age6','X8']
반응변수= ['Y']

###  모형 선택

In [56]:
pd_trainX = pd_train[설명변수]
pd_trainY = pd_train[반응변수]

pd_testX = pd_test[설명변수]
pd_testY = pd_test[반응변수]

#### Decision Tree 형성 후 정확도 예측

In [57]:
### 1) 모델 생성 Decision Tree 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


X_train, X_test, y_train, y_test = train_test_split(pd_trainX,pd_trainY,test_size=0.3, random_state=11)
dt_model = DecisionTreeClassifier(max_depth=5 ,random_state=156)
dt_model.fit(X_train,y_train)
dt_pred = dt_model.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

DecisionTreeClassifier 정확도: 0.8575


In [58]:
##2) 테스트 데이터 pd_test로 모델의 정확도 살펴보기
prediction_list= dt_model.predict(pd_testX)
print(prediction_list)
print('\n')

dt_pred = dt_model.predict(pd_testX)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(pd_testY, dt_pred)))

[1 1 1 ... 1 0 0]


DecisionTreeClassifier 정확도: 0.8550


In [59]:
### 최종 모델의 유의성 검정
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, accuracy_score

model = sm.Logit(pd_trainY, sm.add_constant(pd_trainX))  
results = model.fit()   
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.109429
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:                      Y   No. Observations:               940079
Model:                          Logit   Df Residuals:                   940070
Method:                           MLE   Df Model:                            8
Date:                Fri, 16 Jun 2023   Pseudo R-squ.:                  0.8421
Time:                        20:13:56   Log-Likelihood:            -1.0287e+05
converged:                       True   LL-Null:                   -6.5161e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         37.0849      0.131    282.391      0.000      36.828      37.342
X2             7.3499      0

In [60]:
muhat = results.predict(sm.add_constant(pd_testX))
print("예측확률:")
print(muhat[0:5])
yhat = (muhat > 0.5).astype(int)
print("예측분류:")
print(yhat[0:5])
cm = confusion_matrix(pd_testY, yhat) 
print ("혼돈행렬 : \n", cm) 

예측확률:
0    1.000000
1    0.034598
2    1.000000
3    0.855686
5    0.246849
dtype: float64
예측분류:
0    1
1    0
2    1
3    1
5    0
dtype: int32
혼돈행렬 : 
 [[19597  1021]
 [  975 19833]]
